In [1]:
# ray Roberta lime
from explainability_techniques.lime import LIMEExplainer
import pandas as pd
import pickle
from torch import nn
from models.roberta_model import classify_comment
from transformers import AutoTokenizer
from utils.utils import load_dataset, save_explanation
import sys
sys.path.append(
    'D:/Universidad/informatica 4 año/2do Semestre/tesis_XAI')

In [2]:
dataset = load_dataset(
    "../models/relevance_classification/dataset/facebook_labeled.csv", sep=",")

In [3]:
from models.roberta_model import load_roberta_model_notebook


roberta_model = load_roberta_model_notebook()

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

D:\Universidad/informatica 4 año/2do Semestre/tesis_XAI\models\roberta_model.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(
c:\Users\naybv\AppData\Lo

In [4]:

class_names = ['not relevant', 'relevant']
explainer_roberta = LIMEExplainer(class_names=class_names)

In [5]:
instance_index = 1
instance_roberta = dataset['Review'].iloc[instance_index]
print(instance_roberta)

i have to go to the web for that and i cant organize my photos in certain albums only in the web!



In [6]:
explanation_roberta = explainer_roberta.explain_instance(
    instance_roberta, classify_comment, num_features=20, num_samples=100)

D:\Universidad/informatica 4 año/2do Semestre/tesis_XAI\models\roberta_model.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(


In [9]:
# Extraer las probabilidades de predicción generadas por LIME
lime_probabilities = explainer_roberta.get_predict_proba(
    explanation_roberta)  # Probabilidades generadas por LIME
print(lime_probabilities)

[0.00959295 0.99040705]


In [10]:
# Preparar los datos para el Excel
data = []
# Obtener la lista de rasgos y pesos de la explicación
explanation_list = explanation_roberta.as_list()
for i, class_name in enumerate(class_names):
    # Filtrar los rasgos y valores de pertenencia según la clase
    if class_name == 'not relevant':
        relevant_features = [feature for feature,
                             weight in explanation_list if weight < 0]
        relevant_weights = [weight for feature,
                            weight in explanation_list if weight < 0]
    else:  # class_name == 'relevant'
        relevant_features = [feature for feature,
                             weight in explanation_list if weight > 0]
        relevant_weights = [weight for feature,
                            weight in explanation_list if weight > 0]

    data.append({
        'Instancia': instance_roberta,
        'Clase': class_name,
        # Probabilidad asignada por LIME a esta clase
        'Probabilidad (LIME)': lime_probabilities[i],
        # Convertir lista en string para mejor visualización en Excel
        'Rasgos': ', '.join(relevant_features),
        # Convertir lista en string
        'Valores de Pertenencia': ', '.join(map(str, relevant_weights))
    })

# Crear un DataFrame de pandas
df = pd.DataFrame(data)

# Guardar el DataFrame en un archivo Excel
excel_file = f'lime_explanation__{instance_index}.xlsx'
df.to_excel(excel_file, index=False)

print(f"Explicación guardada en {excel_file}")

Explicación guardada en lime_explanation__1.xlsx


In [11]:
html_roberta = explanation_roberta.as_html()
save_explanation(html_roberta, 'roberta_explanation_opinion573_100_lime.html')
explainer_roberta.save_and_open_html(
    html_roberta, 'roberta_explanation_opinion573_100_lime.html')